# LigthGBM - CLASSIFICATION - BAYESIAN OPTIMIZATION

In [1]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
import lightgbm as lgb
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
import pandas as pd
from datetime import datetime
from time import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [3]:
DATAPATH = 'data/train_test/'
SEED = 47
NITER = 100
CV = 3
SCORE = 'roc_auc'
handlingnull = False
NJOBS = -1
USEGPU = False
NCLASS = 3 # number class to predict (if bivar set 0)

### LOAD DATASET

In [10]:
train_features = pd.read_pickle(DATAPATH+'X_train.pkl')

In [11]:
train_labels = pd.read_pickle(DATAPATH+'y_train.pkl')['target']

In [12]:
features_eng = np.load('data/train_test/features_selected.npy')

In [13]:
train_features = train_features[features_eng]

In [14]:
train_features.shape

(148865, 800)

In [15]:
train_labels.shape

(148865,)

In [16]:
train_features = train_features.values

In [17]:
train_labels = train_labels.values

#### Input Null values

In [18]:
if handlingnull:
    train_features[np.isnan(train_features)] = -9999

In [19]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(train_features, train_labels)

In [20]:
del(train_features)

### TRAIN MODEL

#### Set Search hyperparameters

In [21]:
# ======== General Parameters ======= #

# Select the type of model to run at each iteration. gbtree or gblinear.
boosting = 'gbdt'


# ======== Booster Parameters ======== # 

# Analogous to learning rate in GBM. 
# Typical final values to be used: 0.01-0.2
eta = [0.01] 


# A node is split only when the resulting split gives a positive reduction in the loss function. 
# Gamma specifies the minimum loss reduction required to make a split.
gamma = [i/10.0 for i in range(0,5)]


# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = (len(train_labels) - sum(train_labels))/sum(train_labels)


# Learning Task Parameters
# This defines the loss function to be minimized. See documentation
# -  options: regression, regression_l1, huber, fair, poisson, quantile, 
# mape, gamma, tweedie, binary, multiclass, multiclassova, cross_entropy, cross_entropy_lambda,
# lambdarank, aliases: objective_type, app, application
objective  = 'binary'


# The metric to be used for validation data.
# - rmse, root square loss, aliases: root_mean_squared_error, l2_root
# - quantile, Quantile regression
# - mape, MAPE loss, aliases: mean_absolute_percentage_error
# - huber, Huber loss
# - fair, Fair loss
# - poisson, negative log-likelihood for Poisson regression
# - gamma, negative log-likelihood for Gamma regression
# - gamma_deviance, residual deviance for Gamma regression
# - tweedie, negative log-likelihood for Tweedie regression
# - ndcg, NDCG, aliases: lambdarank
# - map, MAP, aliases: mean_average_precision
# - auc, AUC
# - binary_logloss, log loss, aliases: binary
metric = 'auc'

[lightGBM params](https://lightgbm.readthedocs.io/en/latest/Parameters.html)


help(lgb.LGBMClassifier)

In [22]:
# Define the search space
# Domain space-- Range of hyperparameters
pds = {
    # Minimal number of data in one leaf. Can be used to deal with over-fitting
    # default = 20, type = int, aliases: min_data_per_leaf, min_data, min_child_samples, 
    'num_leaves': (20, 100),

    # Denotes the fraction of columns to be randomly samples for each tree.
    'feature_fraction': (0.1, 0.9),
    
    # Denotes the fraction of observations to be randomly samples for each tree.
    'bagging_fraction': (0.8, 1),

    # The maximum depth of a tree
    'max_depth': (9, 13 ),

    'min_split_gain': (0.001, 0.1),

    # Minimal sum hessian in one leaf. Like min_data_in_leaf, it can be used to deal with over-fitting
    # Default 1e-3
    'min_child_weight': (30, 50),
    
   

    
}

In [23]:
pds

{'num_leaves': (20, 100),
 'feature_fraction': (0.1, 0.9),
 'bagging_fraction': (0.8, 1),
 'max_depth': (9, 13),
 'min_split_gain': (0.001, 0.1),
 'min_child_weight': (30, 50)}

#### Find num boost

In [24]:
early_stopping_rounds = 50

In [25]:
# specify your configurations as a dict
lgb_params = {
    'boosting_type': boosting,
    'objective': objective,
    'metric': metric,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_threads' : 8,
    'verbose': 0,
    #'num_class':  NCLASS,
    'seed' : SEED
}

In [26]:
cvresult = lgb.cv(lgb_params, lgb_train, num_boost_round = 1000, nfold = CV, metrics = metric, early_stopping_rounds = early_stopping_rounds, seed = SEED)

In [27]:
n_estimators = pd.DataFrame(cvresult).shape[0]

In [29]:
print("Best number of estimators found: ", n_estimators)

Best number of estimators found:  344


#### Bayesian optimization hyperparameters

In [30]:
def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight):
      
    params = {'boosting_type': boosting,
              'application': objective,
              'num_iterations': n_estimators,
              'learning_rate':eta, 
              'early_stopping_round':50,
              'metric': metric} # Default parameters
    
    
    params["num_leaves"] = int(round(num_leaves))
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    params['min_split_gain'] = min_split_gain
    
    
    
    cv_result = lgb.cv(params, lgb_train, nfold=CV, seed=SEED, stratified=False, verbose_eval=None, metrics = metric)
    
    # Extract the best score
    best_score = max(cv_result['auc-mean'])
    
    # Loss must be minimized
    #loss = 1 - best_score

    return best_score

In [31]:
# Surrogate model
optimizer = BayesianOptimization(hyp_lgbm,pds,random_state=7)

In [32]:
# Optimize
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | baggin... | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7963   |  0.8153   |  0.7239   |  10.75    |  44.47    |  0.09782  |  63.08    |
|  2        |  0.7922   |  0.9002   |  0.1576   |  10.07    |  40.0     |  0.06824  |  84.3     |
|  3        |  0.7906   |  0.8762   |  0.1527   |  10.15    |  48.19    |  0.02213  |  56.17    |
|  4        |  0.7886   |  0.9862   |  0.1199   |  11.4     |  49.0     |  0.0238   |  63.88    |
|  5        |  0.7931   |  0.9818   |  0.2065   |  11.09    |  45.01    |  0.06723  |  57.42    |
|  6        |  0.7913   |  0.811    |  0.897    |  12.87    |  30.29    |  0.008335 |  20.08    |
|  7        |  0.7955   |  0.8867   |  0.8468   |  9.032    |  30.22    |  0.09413  |  55.01    |
|  8        |  0.7979   |  0.849    |  0.8811   |  12.96    |  30.0     |  0.09479  |  98.65    |
|  9        |  0.791

#### Best parameter

In [38]:
best_params = optimizer.max['params']

In [39]:
best_params['n_estimators'] = n_estimators

#### Saving best hyperparameters

In [40]:
np.save('output/models/bayesianoptcv_gbm_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', best_params)